In [4]:
import os
import ctypes
import vlc
import time
import keyboard

# Set the path to the VLC installation directory
vlc_path = r'C:/Program Files/VideoLAN/VLC'

# Ensure the VLC path exists
if not os.path.exists(vlc_path):
    raise FileNotFoundError(f"VLC path not found: {vlc_path}")

# Add the VLC directory to the DLL search path
os.add_dll_directory(vlc_path)

# Set the full path to the libvlc.dll file
libvlc_dll = os.path.join(vlc_path, 'libvlc.dll')

# Load the libvlc.dll with error handling
try:
    ctypes.CDLL(libvlc_dll)
except OSError as e:
    raise OSError(f"Error loading libvlc.dll: {e}")

# Create an instance of VLC
instance = vlc.Instance()
player = instance.media_player_new()
webcam_device = 'dshow://'  # For Windows
media = instance.media_new(webcam_device)
player.set_media(media)

# Start playing the webcam stream
player.play()

# Keep the stream open and stop when 'q' is pressed
try:
    print("Press 'q' to quit")
    while True:
        if keyboard.is_pressed('q'):
            print("Quitting...")
            break
        time.sleep(1)
except KeyboardInterrupt:
    pass

# Stop the media player
player.stop()


Press 'q' to quit


In [3]:
import cv2
import vlc
import time
import threading
from datetime import datetime
import sys

# RTSP stream URLs (example)
rtsp_urls = [
    'rtsp://username:password@ip_address_1:port/stream',
    'rtsp://username:password@ip_address_2:port/stream',
    'rtsp://username:password@ip_address_3:port/stream'
]

# Output files for recording
output_files = ['output1.avi', 'output2.avi', 'output3.avi']

# Video capture objects for webcams
caps = []

# VLC media players for RTSP streams
vlc_players = []

# Recording state
recording = [False] * len(rtsp_urls)
recording_threads = [None] * len(rtsp_urls)

def initialize_captures(use_rtsp):
    global caps, vlc_players
    caps = []
    vlc_players = []

    if use_rtsp:
        for url in rtsp_urls:
            player = vlc.MediaPlayer(url)
            vlc_players.append(player)
            player.play()
            print(f"Initialized VLC player for: {url}")
    else:
        for i in range(len(rtsp_urls)):
            cap = cv2.VideoCapture(0 + i)  # Assuming multiple webcams
            if cap.isOpened():
                print(f"Opened webcam {i}")
            else:
                print(f"Failed to open webcam {i}")
            caps.append(cap)

def start_recording(index):
    global recording
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for .avi
    out = cv2.VideoWriter(output_files[index], fourcc, 20.0, (640, 480))  # Adjust frame size as needed

    while recording[index]:
        ret, frame = caps[index].read()
        if ret:
            out.write(frame)
        time.sleep(1 / 20)  # Adjust based on desired frame rate

    out.release()
    print(f"Recording stopped for stream {index + 1}. Video saved as {output_files[index]}")

def stream_and_display(index, use_rtsp):
    global caps, vlc_players
    while True:
        if use_rtsp:
            time.sleep(0.1)  # VLC handles playback internally
        else:
            ret, frame = caps[index].read()
            if ret:
                cv2.imshow(f'Webcam {index + 1}', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

def toggle_recording(index):
    global recording
    if recording[index]:
        recording[index] = False  # Stop the recording
        if recording_threads[index] is not None:
            recording_threads[index].join()  # Wait for the thread to finish
    else:
        recording[index] = True  # Start the recording
        recording_threads[index] = threading.Thread(target=start_recording, args=(index,))
        recording_threads[index].start()
        print(f"Recording started for stream {index + 1}")

def main(start_time=None, end_time=None, use_rtsp=True):
    # Initialize captures based on the feed type
    initialize_captures(use_rtsp)

    # Start the streams and display them
    for i in range(len(rtsp_urls)):
        threading.Thread(target=stream_and_display, args=(i, use_rtsp)).start()

    # Control for recording
    try:
        print("Press 'r' to start/stop recording for all streams, 'q' to quit")
        while True:
            current_time = datetime.now().strftime('%H:%M')
            if start_time and current_time == start_time:
                for i in range(len(rtsp_urls)):
                    toggle_recording(i)  # Start recording at the specified start time
                time.sleep(60)  # Sleep for a minute to avoid multiple triggers

            if end_time and current_time == end_time:
                for i in range(len(rtsp_urls)):
                    recording[i] = False  # Stop recording at the specified end time
                print("All recordings stopped.")
                break

            # Key press control
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("Quitting...")
                #break
                sys.exit()
            if key == ord('r'):
                for i in range(len(rtsp_urls)):
                    toggle_recording(i)

    except KeyboardInterrupt:
        pass

    # Cleanup
    for cap in caps:
        cap.release()
    cv2.destroyAllWindows()

    for player in vlc_players:
        player.stop()

# Example usage
if __name__ == '__main__':
    main(start_time='14:30', end_time='14:35', use_rtsp=False)  # Set your desired start and end times here


Opened webcam 0
Failed to open webcam 1
Failed to open webcam 2
Press 'r' to start/stop recording for all streams, 'q' to quit
